# RAG Demo Pipeline using e5-base + Ollama

This demo loads the REST API PDF from `data/`, builds a FAISS index, answers a question and evaluates with RAGAS metrics and Cosine Similarity


In [ ]:
# Setup: Configure paths and imports
import sys
import os
from pathlib import Path

# Set working directory to project root
project_root = Path.cwd().parent
os.chdir(project_root)
sys.path.insert(0, str(project_root))


In [ ]:
# Import RAG pipeline modules and dependencies
from src.ingest import extract_text, chunk_text
from src.embeddings import load_e5, embed_passages, embed_query
from src.vector_store import VectorStore
from src.qa import LocalLLMQA
from src.evaluate import evaluate_ragas, evaluate_cosine_similarity
import pandas as pd
import json


/Users/adamhakeem17/Desktop/Evaluation_Project_SCC/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load REST API Documentation

In [ ]:
# Ingest and chunk the PDF document
data = Path('data/DSP_API_Guide-200-en.pdf')
raw_text = extract_text(data)
chunks = chunk_text(raw_text)  # Default: chunk_size=1024, overlap=200

print(f"Loaded {data.name} with {len(chunks)} chunks")


Loaded DSP_API_Guide-200-en.pdf with 52 chunks


### Load Ground Truths

In [ ]:
# Load ground truth question-answer pairs for evaluation
gts = json.load(open("data/ground_truth.json"))
questions = [item["question"] for item in gts["items"]]
ground_truths = [item["answer"] for item in gts["items"]]

print(f"Loaded {len(questions)} question-answer pairs for evaluation")


### Build Index

In [ ]:
# Build vector index
# Load e5-base embedding model
model = load_e5()

# Embed all chunks using e5-base with "passage:" prefix
emb = embed_passages(model, chunks)

# Create FAISS index and add embeddings
store = VectorStore(emb.shape[1])
store.add(emb, chunks)

print(f"Built FAISS index with {len(chunks)} embeddings")


### Ask a question

In [ ]:
# Test RAG pipeline with a sample question
# Initialize local LLM (Qwen 2.5 1.5B via Ollama)
qa = LocalLLMQA()

# Test with a question, including one outside the document to test "I don't know" behavior
q = 'What is the Capital of France?'
retrieved = store.search(embed_query(model, q))
contexts = [c[0] for c in retrieved]
answer = qa.answer(q, contexts)

print("Answer:")
print(answer)
print("\nRetrieved Contexts:")
for i, context in enumerate(contexts, 1):
    print(f"\n[{i}] {context[:150]}...")  # Show first 150 chars of each context


Answer:
I don't know. The context provided does not contain any information about the capital city of France.

Retrieved Contexts:

[1] EST APIs ........................................................................ 5
2.1. The Platform REST APIs .........................................

[2] on that offers a better
understanding of Swagger UI for any casual user.
Page 4 of 26
REST API User Guide
2. Introduction to the Platform REST APIs
Th...

[3] ser Guide
The schema is a document that’s generated from the Open API specification. It gives a
useful shape to the API, and is an additional quick re...

[4] ss Information
• Attributes
• Data Classes
• Tags
• Terms
Data Exchange
• Assets
• Connections
• Data Planes
• Datasets
• Fields
Policies
• Access Con...

[5] c to data
classes and transformations.
Users of the platform can utilize the following types of rules:
• access control rules
• transformation rules
P...


### Evaluation

In [ ]:
# Generate answers for all evaluation questions
answers = []
contexts_list = []

for q in questions:
    # Retrieve top-5 relevant chunks
    retrieved = store.search(embed_query(model, q))
    contexts = [c[0] for c in retrieved]
    contexts_list.append(contexts)
    
    # Generate answer using local LLM with retrieved context only
    answers.append(qa.answer(q, contexts))

# Compute cosine similarity between answers and ground truths
cosine_scores = evaluate_cosine_similarity(answers, ground_truths)

cos_df = pd.DataFrame({
    "cosine_similarity": cosine_scores,
})

print(f"Generated {len(answers)} answers. Cosine similarity scores computed.")
cos_df


,cosine_similarity
0,0.895983
1,0.955409
2,0.917767
3,0.934184
4,0.900762
5,0.927240
6,0.910291
7,0.934118
8,0.897515
9,0.918441


In [ ]:
# Run RAGAS evaluation (faithfulness + answer relevancy)
# RAGAS uses a local LLM to evaluate both metrics
ragas_result = evaluate_ragas(
    questions,
    answers,
    contexts_list,
    qa.llm,  # Use the same local LLM (Qwen 2.5 1.5B)
    ground_truths,
)

ragas_df = ragas_result.to_pandas()

# Rename RAGAS columns for clarity
ragas_df = ragas_df.rename(columns={
    'answer_relevancy': 'Answer Relevancy',
    'faithfulness': 'Faithfulness',
    'user_input': 'Question',
    'response': 'LLM Response',
    'retrieved_contexts': 'Retrieved Context',
    'reference': 'Ground Truth'
})

print("RAGAS evaluation complete.")
ragas_df


Evaluating: 100%|██████████| 20/20 [03:06<00:00,  9.33s/it]


,Question,Retrieved Context,LLM Response,Ground Truth,Faithfulness,Answer Relevancy
0,How do I generate a JWT bearer token to authen...,[ken into the\nheader. The host server must fi...,To generate a JWT bearer token for authenticat...,Issue a POST request to /graph/enterprise-mana...,0.00,0.965416
1,What is the base URL structure for all API end...,[on that offers a better\nunderstanding of Swa...,The base URL structure for all API endpoints i...,All API endpoints follow the structure: https:...,1.00,1.000000
2,How do I find the exchange ID needed for certa...,[o:\nhttps://<your_DSP_environment_url>/api/v1...,To find the exchange ID needed for certain API...,"Log in to the platform, click Data Exchange in...",0.00,1.000000
3,Can I delete system attribute types using the ...,[ss Information\n• Attributes\n• Data Classes\...,"Yes, you can delete system attribute types usi...","No, you cannot delete system attribute types, ...",1.00,0.929590
4,What's the difference between submitting an as...,[record being\nqueried (requested) according t...,Creating a draft asset and submitting it for r...,POST /api/v1/datasets/{datasetId}/assets creat...,0.50,0.977714
5,How do I approve or reject a transformation po...,[e 9 of 26\nREST API User Guide\nEndpoint Desc...,To approve or reject a transformation policy t...,Use PUT /api/v1/policy-tasks/{id} to approve o...,1.00,0.976312
6,What HTTP status code indicates successful cre...,[a % symbol at the end (to let you know that t...,The HTTP status code that indicates successful...,A successful POST (create) request returns a 2...,1.00,0.997775
7,Do consumption projects and migration projects...,[.\nPOST/api/v1/transformation-policies/ Creat...,"No, consumption projects and migration project...","No, the Project API endpoints (under /api/v1/p...",0.75,0.899621
8,How can I interactively test API endpoints wit...,[w\nwhether or not the operation was successfu...,You can interactively test API endpoints using...,Use Swagger UI by navigating to https://<your_...,1.00,0.880363
9,What format are all API request and response p...,"[rized as follows:\n• It is ""stateless""—that i...",All API request and response payloads are in J...,All REST API requests and responses use JSON (...,1.00,0.991379


In [ ]:
# Merge evaluation metrics into final results dataframe
eval_df = ragas_df.copy()

# Add cosine similarity scores
eval_df['Cosine Similarity'] = cos_df['cosine_similarity'].values[:len(eval_df)]

print("Final evaluation results:")
eval_df


,Question,Retrieved Context,LLM Response,Ground Truth,Faithfulness,Answer Relevancy,Cosine Similarity
0,How do I generate a JWT bearer token to authen...,[ken into the\nheader. The host server must fi...,To generate a JWT bearer token for authenticat...,Issue a POST request to /graph/enterprise-mana...,0.00,0.965416,0.895983
1,What is the base URL structure for all API end...,[on that offers a better\nunderstanding of Swa...,The base URL structure for all API endpoints i...,All API endpoints follow the structure: https:...,1.00,1.000000,0.955409
2,How do I find the exchange ID needed for certa...,[o:\nhttps://<your_DSP_environment_url>/api/v1...,To find the exchange ID needed for certain API...,"Log in to the platform, click Data Exchange in...",0.00,1.000000,0.917767
3,Can I delete system attribute types using the ...,[ss Information\n• Attributes\n• Data Classes\...,"Yes, you can delete system attribute types usi...","No, you cannot delete system attribute types, ...",1.00,0.929590,0.934184
4,What's the difference between submitting an as...,[record being\nqueried (requested) according t...,Creating a draft asset and submitting it for r...,POST /api/v1/datasets/{datasetId}/assets creat...,0.50,0.977714,0.900762
5,How do I approve or reject a transformation po...,[e 9 of 26\nREST API User Guide\nEndpoint Desc...,To approve or reject a transformation policy t...,Use PUT /api/v1/policy-tasks/{id} to approve o...,1.00,0.976312,0.927240
6,What HTTP status code indicates successful cre...,[a % symbol at the end (to let you know that t...,The HTTP status code that indicates successful...,A successful POST (create) request returns a 2...,1.00,0.997775,0.910291
7,Do consumption projects and migration projects...,[.\nPOST/api/v1/transformation-policies/ Creat...,"No, consumption projects and migration project...","No, the Project API endpoints (under /api/v1/p...",0.75,0.899621,0.934118
8,How can I interactively test API endpoints wit...,[w\nwhether or not the operation was successfu...,You can interactively test API endpoints using...,Use Swagger UI by navigating to https://<your_...,1.00,0.880363,0.897515
9,What format are all API request and response p...,"[rized as follows:\n• It is ""stateless""—that i...",All API request and response payloads are in J...,All REST API requests and responses use JSON (...,1.00,0.991379,0.918441


In [ ]:
# Save evaluation results to CSV
# Create results directory if it doesn't exist
results_dir = Path("results")
results_dir.mkdir(exist_ok=True)

# Save results
eval_df.to_csv(results_dir / "evaluation_results_1024.csv", index=False)

print(f"✓ Results saved to results/evaluation_results_1024.csv")
